In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

# 미니 배치는 기울기(gradient)를 효율적으로 계산하고 업데이트하기 위해 전체 데이터셋에서 추출하는 작은 데이터 묶음.
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits # 활성화 함수(activation function)를 거치기 전의 결과 값

model = NeuralNetwork()

100%|██████████| 26.4M/26.4M [00:00<00:00, 114MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 5.98MB/s]
100%|██████████| 4.42M/4.42M [00:00<00:00, 61.0MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 19.2MB/s]


In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [4]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

In [5]:
# 옵티마이저를 쓰는 이유는 모델의 손실을 최소화하는 최적의 가중치(parameters)를 효율적으로 찾아내기 위함.
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

#  옵티마이저가 찾아낸 가중치(parameters)를 가지고 경사하강법을 통해 손실 함수 값을 줄이는 방향으로 가중치를 업데이트합니다.

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) # 전체 데이터셋의 총 샘플 수를 구하는 역할
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X) # batch * batch_size + len(X) -> 직전 배치까지 처리한 샘플의 총 개수
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad(): # 모델을 평가할 때는 가중치를 바꿀 필요가 없습니다. 따라서 기울기 계산을 생략함으로써 메모리를 절약하고 연산 속도를 크게 높일 수 있습니다.
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

pred.argmax(1): pred 텐서는 모델이 각 클래스에 대해 예측한 원시 점수(로짓)를 담고 있습니다. 예를 들어, 3개의 클래스 분류 문제에서 한 샘플에 대한 예측값이 [0.2, 1.5, -0.1]라면, argmax(1)은 가장 큰 값인 1.5의 인덱스인 1을 반환합니다. 이 결과는 모델이 예측한 클래스를 의미합니다.

== y: pred.argmax(1)이 반환한 예측 클래스(인덱스)와 실제 정답 레이블인 y를 비교합니다. 이 비교 결과는 True 또는 False로 이루어진 불리언 텐서가 됩니다. 예측이 맞으면 True, 틀리면 False가 되는 식입니다.

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.298171  [   64/60000]
loss: 2.291299  [ 6464/60000]
loss: 2.271887  [12864/60000]
loss: 2.264206  [19264/60000]
loss: 2.240529  [25664/60000]
loss: 2.225499  [32064/60000]
loss: 2.225888  [38464/60000]
loss: 2.196253  [44864/60000]
loss: 2.187274  [51264/60000]
loss: 2.166602  [57664/60000]
Test Error: 
 Accuracy: 52.5%, Avg loss: 2.158272 

Epoch 2
-------------------------------
loss: 2.167623  [   64/60000]
loss: 2.155992  [ 6464/60000]
loss: 2.100586  [12864/60000]
loss: 2.112429  [19264/60000]
loss: 2.053012  [25664/60000]
loss: 2.015520  [32064/60000]
loss: 2.030861  [38464/60000]
loss: 1.958388  [44864/60000]
loss: 1.956955  [51264/60000]
loss: 1.898381  [57664/60000]
Test Error: 
 Accuracy: 54.6%, Avg loss: 1.890497 

Epoch 3
-------------------------------
loss: 1.918789  [   64/60000]
loss: 1.888941  [ 6464/60000]
loss: 1.774575  [12864/60000]
loss: 1.813228  [19264/60000]
loss: 1.701374  [25664/60000]
loss: 1.671389  [32064/600